In [2]:
from sentence_transformers import SentenceTransformer,util
import pandas as pd
import torch

/home/max/anaconda3/envs/kaggle/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def group_and_join_by_time(start_times, text_column, time_interval=5):
    grouped_text = []
    current_group = []
    current_time = 0

    for start, text in zip(start_times, text_column):
        if start - current_time > time_interval:
            grouped_text.append(' '.join(current_group))
            current_group = []
            current_time = start
        current_group.append(text)

    # Add the last group
    if current_group:
        grouped_text.append(' '.join(current_group))

    return grouped_text

In [4]:
df = pd.read_json('df.json',lines=True)

# Apply the function to group and join text every 5 seconds
grouped_text_by_time = df.groupby(['video_id','video_title']).apply(lambda group: group_and_join_by_time(group['start'], group['text']))

expanded_texts = []
expanded_start_times = []
video_ids = []
video_titles = []


for value, sentences in grouped_text_by_time.items():
    video_id = value[0]
    video_title = value[1]

    start_times = list(range(0, len(sentences) * 5, 5))
   # Extend the lists with the expanded data
    expanded_texts.extend(sentences)
    expanded_start_times.extend(start_times)
    video_ids.extend([video_id] * len(sentences))
    video_titles.extend([video_title] * len(sentences))  # Extend with the corresponding video_title

# Create DataFrame from the expanded data
df_expanded = pd.DataFrame({'video_id': video_ids,'video_title':video_titles, 'start_time': expanded_start_times, 'text': expanded_texts})

model_semantic_search = SentenceTransformer("all-MiniLM-L6-v2")

# Sentences are encoded by calling model.encode()
passage_embedding = model_semantic_search.encode(df_expanded.iloc[0:3]['text'].tolist())

df_expanded['passage_embedding'] = passage_embedding.tolist()

/tmp/ipykernel_671379/4287534716.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_text_by_time = df.groupby(['video_id','video_title']).apply(lambda group: group_and_join_by_time(group['start'], group['text']))
